# SYCL Hello World

This code creates a SYCL program that runs a simple “Hello World!” task on a GPU. By using `sycl::gpu_selector_v`, it ensures that the task is submitted specifically to a GPU if one is available. This example is great for demonstrating basic task execution on a SYCL device and how to use the `sycl::stream` feature to output from within the kernel.

In [ ]:
%%writefile compute.cpp
#include <iostream>
#include <sycl/sycl.hpp>

int main() {
    
    auto myQueue = sycl::queue{sycl::gpu_selector_v};
    std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << std::endl;

    // Submit a command group to the queue
    myQueue.submit([&](sycl::handler &cgh) {
                   
            // Create a stream for output within kernel
            auto os = sycl::stream{128, 128, cgh};

        //              _             _        _            _
        //          ___(_)_ __   __ _| | ___  | |_ __ _ ___| | __
        //         / __| | '_ \ / _` | |/ _ \ | __/ _` / __| |/ /
        //         \__ \ | | | | (_| | |  __/ | || (_| \__ \   <
        //         |___/_|_| |_|\__, |_|\___|  \__\__,_|___/_|\_\
        //                      |___/

            // Execute a single task
            cgh.single_task([=]() {
                os << "Hello World!"<< sycl::endl; 
                });
        })
        .wait();  // Wait for completion
    return 0;
}

The cell below creates a shell script that sets up the necessary environment for SYCL, compiles the `compute.cpp` file, and executes it if the compilation is successful.

In [ ]:
%%writefile ./run-sycl.sh
#!/bin/bash 
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

The final cell below makes the script executable and runs it, executing the SYCL program that prints “Hello World!” from the GPU.

In [ ]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh